<a id= 'top'></a>


# Import JASON file, clean text to csv:

In the following note book the generated JASON file from mongoDB database is initially imprted to a dataframe and the available text was then processed for visualization and sentimental analysis tasks.   

There is a great [Kaggle tutorial](https://www.kaggle.com/c/word2vec-nlp-tutorial/details/part-1-for-beginners-bag-of-words) on Natural Language Processing (NLP) and sentimental analysis. 






In [1]:
# Importing the needed libraries 

import pandas as pd
from pandas import DataFrame, Series
from pandas.tseries.resample import TimeGrouper
from pandas.tseries.offsets import DateOffset
import json
import matplotlib.pyplot as plt
%matplotlib inline 

from IPython.display import HTML
import folium
from wordcloud import WordCloud, STOPWORDS

from scipy.misc import imread

import nltk
from nltk.corpus import stopwords

from bs4 import BeautifulSoup
import re


## Reading JASON file

Importing the mongoDB generated JASON file in to a DataFram. 

 [Return to top](#top)

In [2]:
# Reading mongodb exported SB50-Cambridge.json file in to a panda DataFrame
sb50 = DataFrame([json.loads(line) for line in open('../Data/SB50-Cambridge.json')])

In [3]:
sb50.tail()

,_id,created_at,geo,source,text
2046247,{'$oid': '56b87a70a9a2f606fe893a60'},{'$date': '2016-02-08T11:22:24.000Z'},None,Twitter for iPhone,RT @coopjet06: When you live in England so the...
2046248,{'$oid': '56b87a70a9a2f606fe893a61'},{'$date': '2016-02-08T11:22:24.000Z'},None,Facebook,The important message behind Beyoncé's Super B...
2046249,{'$oid': '56b87a70a9a2f606fe893a62'},{'$date': '2016-02-08T11:22:24.000Z'},None,Twitter Web Client,اسوي لك بحوث جامعية و مشروع تخرج واتس 00974700...
2046250,{'$oid': '56b87a70a9a2f606fe893a63'},{'$date': '2016-02-08T11:22:24.000Z'},None,Twitter for Windows Phone,RT @FederiKoRaivan: Homophobes hating on lady ...
2046251,{'$oid': '56b87a71a9a2f606fe893a64'},{'$date': '2016-02-08T11:22:24.000Z'},None,Flipboard,"RT @empiremagazine: ICYMI: That’s it, it’s war..."


In [4]:
sb50['Created'] = pd.read_csv('../Data/datestamp.csv')
sb50.head()

,_id,created_at,geo,source,text,Created
0,{'$oid': '56b79c1ea9a2f606245e037d'},{'$date': '2016-02-07T19:33:49.000Z'},None,Twitter for iPhone,Kinda wonder if Beyoncé will bring up Lady Gag...,2016-02-07T19:33:49.000Z
1,{'$oid': '56b79c1ea9a2f606245e037e'},{'$date': '2016-02-07T19:33:49.000Z'},None,Twitter for iPad,Orange Crush! I’m taking the #Broncos to win S...,2016-02-07T19:33:49.000Z
2,{'$oid': '56b79c1ea9a2f606245e037f'},{'$date': '2016-02-07T19:33:49.000Z'},None,Twitter for iPhone,Turns out the grass is greener on that side:Se...,2016-02-07T19:33:49.000Z
3,{'$oid': '56b79c1ea9a2f606245e0380'},{'$date': '2016-02-07T19:33:50.000Z'},None,Twitter for iPhone,@MKBHD don't worry one day the Super Bowl will...,2016-02-07T19:33:50.000Z
4,{'$oid': '56b79c1ea9a2f606245e0381'},{'$date': '2016-02-07T19:33:50.000Z'},None,Kindle,The Super Bowl’s greatest plays\nhttps://t.co/...,2016-02-07T19:33:50.000Z


##  Cleanign the "text"  and "source" Columns
 [Return to top](#top)

In [5]:
def clean (raw_text):

    
#1. Remove HTML
    review_text = BeautifulSoup(raw_text, 'html5lib').get_text() 

# 2. Remove non-letters and Remove website addresses       
    letters_only = re.sub(r"http\S+", "",review_text, flags=re.MULTILINE)
    letters_only = re.sub("[^a-zA-Zé]", " ", letters_only) 
    
# 3. Convert to lower case and split into individual words
    words = letters_only.lower().split()                             
    
# 4. In Python, searching a set is much faster than searching a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  

# 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops 
                            and "'" not in w  
                            and not w.startswith('@')
                            and w != 'rt']   


# 6. Join the words back into one string separated by space and return the result.
    return( " ".join( meaningful_words ))   

In [6]:
clean_tweet = clean( sb50['text'][0] )

print ("Original tweet:", sb50['text'][0])
print ("Clearned tweet:", clean_tweet)

Original tweet: Kinda wonder if Beyoncé will bring up Lady Gaga to preform their song together? https://t.co/fIJO9XcHPO
Clearned tweet: kinda wonder beyonc bring lady gaga preform song together


In [7]:
num_tweets = sb50["text"].size
num_tweets

2046252

In [8]:
clean_tweets = []

# Loop over each review; create an index i that goes from 0 to the length
# of the movie review list 
for i in range( 0, num_tweets ):
    # Call our function for each one, and add the result to the list of
    # clean reviews
    clean_tweets.append(clean( sb50["text"][i] ) )
    
clean_source = []

# Loop over each review; create an index i that goes from 0 to the length
# of the movie review list 
for i in range( 0, num_tweets ):
    # Call our function for each one, and add the result to the list of
    # clean reviews
    clean_source.append(clean( sb50["source"][i] ) )

/anaconda/anaconda3/anaconda/lib/python3.5/site-packages/bs4/__init__.py:207: UserWarning: "b'https://t.co/wsR6e4bvau'" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
/anaconda/anaconda3/anaconda/lib/python3.5/site-packages/bs4/__init__.py:207: UserWarning: "b'https://t.co/5wKDrS1jyC'" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
/anaconda/anaconda3/anaconda/l

In [9]:
sb50['cleantweets'] = clean_tweets
sb50['cleansource'] = clean_source
sb50.tail()

,_id,created_at,geo,source,text,Created,cleantweets,cleansource
2046247,{'$oid': '56b87a70a9a2f606fe893a60'},{'$date': '2016-02-08T11:22:24.000Z'},None,Twitter for iPhone,RT @coopjet06: When you live in England so the...,2016-02-08T11:22:24.000Z,coopjet live england super bowl dosent start t...,twitter iphone
2046248,{'$oid': '56b87a70a9a2f606fe893a61'},{'$date': '2016-02-08T11:22:24.000Z'},None,Facebook,The important message behind Beyoncé's Super B...,2016-02-08T11:22:24.000Z,important message behind beyonc super bowl outfit,facebook
2046249,{'$oid': '56b87a70a9a2f606fe893a62'},{'$date': '2016-02-08T11:22:24.000Z'},None,Twitter Web Client,اسوي لك بحوث جامعية و مشروع تخرج واتس 00974700...,2016-02-08T11:22:24.000Z,dm super bowl,twitter web client
2046250,{'$oid': '56b87a70a9a2f606fe893a63'},{'$date': '2016-02-08T11:22:24.000Z'},None,Twitter for Windows Phone,RT @FederiKoRaivan: Homophobes hating on lady ...,2016-02-08T11:22:24.000Z,federikoraivan homophobes hating lady gaga hal...,twitter windows phone
2046251,{'$oid': '56b87a71a9a2f606fe893a64'},{'$date': '2016-02-08T11:22:24.000Z'},None,Flipboard,"RT @empiremagazine: ICYMI: That’s it, it’s war...",2016-02-08T11:22:24.000Z,empiremagazine icymi war know xmenapocalypse,flipboard


In [10]:
sb50out = sb50.drop(['_id','created_at','text','source'], axis=1)
sb50out.tail()

,geo,Created,cleantweets,cleansource
2046247,None,2016-02-08T11:22:24.000Z,coopjet live england super bowl dosent start t...,twitter iphone
2046248,None,2016-02-08T11:22:24.000Z,important message behind beyonc super bowl outfit,facebook
2046249,None,2016-02-08T11:22:24.000Z,dm super bowl,twitter web client
2046250,None,2016-02-08T11:22:24.000Z,federikoraivan homophobes hating lady gaga hal...,twitter windows phone
2046251,None,2016-02-08T11:22:24.000Z,empiremagazine icymi war know xmenapocalypse,flipboard


In [11]:
sb50out.to_csv('../Data/sb50clean_ver1.csv', index=True)

 [Return to top](#top)